In [1]:
!pip install ultralytics

In [1]:
# Imports
import os, shutil, random, numpy as np, torch, glob, requests, zipfile, tarfile

from ultralytics import YOLO

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.use_deterministic_algorithms(False)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)


In [2]:
registry = {
    "coco2017": {
        "zip": "http://images.cocodataset.org/zips/train2017.zip",
        "yaml": "https://raw.githubusercontent.com/ultralytics/ultralytics/main/ultralytics/cfg/datasets/coco.yaml"
    },

    "openimages_v7": {
        "zip": "https://storage.googleapis.com/openimages/2018_04/train/train-images-boxable.csv",  # subset placeholder
        "yaml": "https://raw.githubusercontent.com/ultralytics/ultralytics/main/ultralytics/cfg/datasets/open-images.yaml"
    },

    "objects365": {
        "zip": "https://huggingface.co/datasets/zhangzhenyong/Objects365/resolve/main/objects365_train.tar",
        "yaml": "https://raw.githubusercontent.com/ultralytics/ultralytics/main/ultralytics/cfg/datasets/objects365.yaml"
    },

    "lvis_v1": {
        "zip": "https://s3-us-west-2.amazonaws.com/dl.fbaipublicfiles.com/lvis/lvis_v1_train.json.zip",
        "yaml": "https://raw.githubusercontent.com/ultralytics/ultralytics/main/ultralytics/cfg/datasets/lvis.yaml"
    },

    "roboflow_universe": {
        "zip": "https://universe.roboflow.com/ds/YOUR_DATASET_DOWNLOAD_LINK",  
        "yaml": "datasets/roboflow_universe.yaml"
    },

    "google_open_buildings": {
        "zip": "https://storage.googleapis.com/open-buildings-data/geojson/buildings.zip",
        "yaml": "datasets/open_buildings.yaml"
    },
    
    "weapon_detection_cctv": {
        "zip": "https://huggingface.co/datasets/Simuletic/cctv-weapon-dataset/resolve/main/data.zip",
        "yaml": "datasets/weapon_detection.yaml"
    },
    "od_weapon_detection": {
        "zip": "https://github.com/ari-dasci/OD-WeaponDetection/archive/refs/heads/main.zip",
        "yaml": "datasets/od_weapon.yaml"
}


In [3]:
# Helpers
def download(url, path):
    r = requests.get(url, stream=True)
    with open(path, "wb") as f:
        for chunk in r.iter_content(8192):
            f.write(chunk)

def extract(path, dst):
    if path.endswith(".zip"):
        with zipfile.ZipFile(path, "r") as z:
            z.extractall(dst)
    elif path.endswith(".tar") or path.endswith(".tar.gz"):
        with tarfile.open(path, "r") as t:
            t.extractall(dst)


In [4]:
# Train function
CHECKPOINT_DIR = "./checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
LATEST = os.path.join(CHECKPOINT_DIR, "latest.pt")

def train(weights, yaml_file, project, name):
    model = YOLO(weights)
    model.train(
        data=yaml_file,
        epochs=20,
        imgsz=640,
        batch=16,
        seed=RANDOM_SEED,
        project=project,
        name=name,
        save=True
    )
    last = glob.glob(f"{project}/{name}/weights/last.pt")[0]
    shutil.copy(last, LATEST)
    return LATEST


In [ ]:
total = len(registry)
count = 0

current_weights = "yolov8n.pt"
if os.path.exists(LATEST):
    current_weights = LATEST

for ds_name, info in registry.items():

    count += 1
    print("\n" + "="*80)
    print(f"📦 Dataset {count}/{total} : {ds_name.upper()}")
    print("="*80)

    zip_path = f"{ds_name}.zip"
    extract_dir = f"dataset_{ds_name}"
    yaml_local = f"yamls/{ds_name}.yaml"

    os.makedirs("yamls", exist_ok=True)

    # Download YAML
    print(f"  Downloading YAML for {ds_name}...")
    download(info["yaml"], yaml_local)

    # Download Dataset
    print(f"  Downloading dataset for {ds_name}...")
    download(info["zip"], zip_path)

    # Extract Dataset
    print(f" Extracting to: {os.path.abspath(extract_dir)}")
    os.makedirs(extract_dir, exist_ok=True)
    extract(zip_path, extract_dir)

    # Progress print
    print(f" Training on {ds_name} (20 epochs)...")

    current_weights = train(
        weights=current_weights,
        yaml_file=yaml_local,
        project=f"train_{ds_name}",
        name=f"run_{ds_name}"
    )

    print(f"✅ Finished {ds_name} ({count}/{total})")



 Dataset 1/6 : COCO2017


In [ ]:
shutil.copy(LATEST, "yolov8n_advanced.pt")
print("\n🎉 Final model saved: yolov8n_advanced.pt")
